In [22]:
import os
import textract
import fitz
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

res_list = []
#res_list =np.array([name][extract_email_addresses])
#res_list.shape()
dir_list = os.listdir(r'D:/FINAL YEAR PROJECT/Resumes/NewTestData/')
resumes = r'D:/FINAL YEAR PROJECT/Resumes/NewTestData/'
c = 0
for i in dir_list:
    c = c + 1
    if c == 5000:
        break
    pfinal = os.path.join(resumes, i)

#def extract_text_from_pdf(file):
    '''Opens and reads in a PDF file from path'''
    resume_text = ""
    if i.endswith('.pdf'):
        doc = fitz.open(pfinal)
        
        for page in doc:
            resume_text = resume_text + str(page.getText())
        resume_text = "".join(resume_text.split("\n"))
    
    '''Opens en reads in a .doc or .docx file from path'''

    if i.endswith('.doc'):
        resume_text = textract.process(pfinal).decode('utf-8')

    job_description = open( "D:/FINAL YEAR PROJECT/Resumes/JobDescriptions/java.txt", "r")
    job_text = job_description.read()
  
    job_description.close()

    my_list = [resume_text, job_text]

    cv = CountVectorizer()
    count_matrix = cv.fit_transform(my_list)

    #Print the similarity scores
    #print("\nSimilarity Scores:")
    #print(cosine_similarity(count_matrix))

    matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
    matchPercentage = round(matchPercentage, 2) # round to two decimal
    #print("Your resume matches about "+ str(matchPercentage)+ "% of the job description.")

    obtainedResumeText = resume_text

    def personalDetailExtractor():
        finalExtractedEmail = []
        finalExtractedPhone = []
        oneFourthOfResume = obtainedResumeText[0:len(obtainedResumeText)//4] 
        emailResume = re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", oneFourthOfResume)
        phoneResume = re.findall(r"(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})",oneFourthOfResume)
        
        if len(emailResume) > 1:
            finalExtractedEmail = emailResume[0]
        else:
            finalExtractedEmail = emailResume
            
        for i in range(len(phoneResume)):
            if len(phoneResume[i])>=10:
                finalExtractedPhone = phoneResume[i]
        return finalExtractedEmail,finalExtractedPhone

    finalExtractedEmail , finalExtractedPhone = personalDetailExtractor()
    #print("Email Address:",finalExtractedEmail)
    #print("Phone Number:",finalExtractedPhone)

    res_list.append(matchPercentage,)
    #print(len(res_list))
    res_list.sort(reverse=True)
    #print(res_list)
    data = [[matchPercentage, finalExtractedEmail, finalExtractedPhone]]

    df = pd.DataFrame(data, columns = ['Percentage', 'Email', 'Phone'])
    print(df)
        
    
    #return txt.replace('\n', ' ').replace('\t', ' ')

   Percentage                    Email Phone
0       34.47  [mikewanaswa@gmail.com]    []
   Percentage                    Email       Phone
0        36.1  [shub.pandey7@yahoo.in]  8115031253
   Percentage                      Email       Phone
0       32.44  [ardraprasad93@gmail.com]  9650459770
   Percentage                 Email       Phone
0       32.72  [satyamce@gmail.com]  9183758878
   Percentage                     Email       Phone
0       25.92  [salman.ali19@gmail.com]  7503729071
   Percentage                  Email       Phone
0       48.98  [viren0312@gmail.com]  9909198882
   Percentage                    Email       Phone
0        36.1  [shub.pandey7@yahoo.in]  8115031253
   Percentage                    Email       Phone
0       44.14  [alamdar1710@gmail.com]  8976297830
   Percentage                     Email       Phone
0       38.83  [ranji90kumar@gmail.com]  8220049406
   Percentage                   Email       Phone
0       44.73  [rmoumita03@gmail.com]  7874038

In [3]:
# -*- coding: utf-8 -*-


#python -m spacy download en_core_web_sm
import spacy
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import wordnet
#from text_process import to_lowercase
from gensim.parsing.preprocessing import remove_stopwords

    #from collections import Counter
    #import en_core_web_sm
dir_list = os.listdir(r'D:/FINAL YEAR PROJECT/Resumes/NewTestData/')
resumes = r'D:/FINAL YEAR PROJECT/Resumes/NewTestData/'
c = 0
for i in dir_list:
    c = c + 1
    if c == 5000:
        break
    resume = os.path.join(resumes, i)

    #Function to extract names from the string using spacy
    def extract_name(resume):
    
    # Load English tokenizer, tagger, parser, NER and word vectors
        #nlp = en_core_web_sm.load()
        nlp = spacy.load('en_core_web_sm')
        doc = nlp(resume)
        for ent in doc.ents:
            if(ent.label_ == 'PER'):
                print(ent.text)
                break 
        return ent.text                  

    #Function to extract Phone Numbers from string using regular expressions
    def extract_phone_numbers(string):
        r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
        phone_numbers = r.findall(string)
        return [re.sub(r'\D', '', number) for number in phone_numbers if len(number) >9]

    #Function to extract Email address from a string using regular expressions
    def extract_email_addresses(string):
        r = re.compile(r'[\w\.-]+@[\w\.-]+')
        return r.findall(string)

    def extractPersonName(tttt, resumeTitle):
            #a = 'Cv_saurabh+keshari_1234_Resume'
            #a = "1234"
            titleSplit = re.split(r'[`\-=~!@#$%^&*()_+\[\]{};\'\\:"|<,./<>?]', resumeTitle)
            title_isNotDigit = []
            for word in titleSplit:
                if not word.isdigit():
                    title_isNotDigit.append(word)
            strr = " ".join(title_isNotDigit)
            strr_list = strr.split(" ")
            #titleSplit = re.findall(r"[\w']+", strr)
            
            Names = []
            for Nouns in strr_list:
                if not wordnet.synsets(Nouns):
                    #Not an English word
                    Names.append(Nouns)
            #titleSplit_lower = to_lowercase(Names)
            titleSplit_cleaned = remove_stopwords(Names)
            personName = "ABC"
            if len(titleSplit_cleaned) > 0:
                if len(titleSplit_cleaned) == 1:
                    firstName = titleSplit_cleaned[0]
                    personName = firstName
                if len(titleSplit_cleaned) == 2: 
                    firstName = titleSplit_cleaned[0]
                    secondName = titleSplit_cleaned[1]
                    personName = firstName +" " + secondName
                if len(titleSplit_cleaned) >= 3: 
                    firstName = titleSplit_cleaned[0]
                    secondName = titleSplit_cleaned[1]
                    thirdName = titleSplit_cleaned[2]
                    personName = firstName +" " + secondName + " " + thirdName
            
            else:
                print("within else")
                TITLE = r"(?:[A-Z][a-z]*\.\s*)?"
                NAME1 = r"[A-Z][a-z]+,?\s+"
                MIDDLE_I = r"(?:[A-Z][a-z]*\.?\s*)?"
                NAME2 = r"[A-Z][a-z]+"
                name = []
                name = re.findall(TITLE + NAME1 + MIDDLE_I + NAME2, tttt)
                if len(name) > 0:
                    rep1 = {"Career": "", "career": "", "Objective":"", "objective": "", "Email": "", "email": "", "Experience Summary": "", "ph": "", "Ph": "", "Professional": "", "Curriculum Vitae": "", "Resume": "", "Profile": "", "Professional": "", "Recruiter": "", "Lead": "", "Summary": "", "HR": "", "new":"", "New": ""}  # define desired replacements here
                    rep = dict((re.escape(k), v) for k, v in rep1.items()) 
                    #Python 3 renamed dict.iteritems to dict.items so use rep.items() for latest versions
                    pattern = re.compile("|".join(rep.keys()))
                    personName = pattern.sub(lambda m: rep[re.escape(m.group(0))], name[0])
                else:  
                    personName = "No name found"
            
            stopwords = ['HR', 'hr','pdf', 'xls', 'docx', 'doc', 'rtf', 'txt', "assembly", "bash", " c " "c++", "c#", "coffeescript", "emacs lisp",
            "go!", "groovy", "haskell", "java", "javascript", "matlab", "max MSP", "objective c", "qlikview", "crm", "CRM", "dynamics",
            "perl", "php","html", "xml", "css", "processing", "python", "ruby", "sml", "swift", "resume","Resume",
            "latex" "unity", "unix" "visual basic" "wolfram language", "xquery", "sql", "node.js", "finance", "liferay",
            "scala", "kdb", "jquery", "mongodb", "CMMI", "ISO", "finance", "Banking", "Finacle", "Oracle Flexcube", "Fiserv", "successfactor", "sf", "success factor", "Success Factor",
            "TCS BaNcs", "FIS Profile", "Self-directed learning", "Collaboration", "Communication", "Resilience", "Big-picture mindset", "Prioritization ", "Creativity ",
            "creative", "Insight", "curiosity", "curious", "Openness", "Teamwork", "Time management", "Emotional intelligence", 
            "quick learner", "problem solver","Customer-service skills", "Planning and organizing", "innovative", "Thinking innovatively and creatively", "Resourceful", "Flexible", "Able to manage own time", "Having self-esteem", 
            "Innovation skills", "Enterprise skills", "Civic or citizenship knowledge and skills", "Sociability", "Self-management", "Integrity", "Honesty", "Human resources", 
            "Participates as a team member", "Works with diversity", "Exercises leadership", "leadership", "Exercises leadership", "Monitors and corrects performance", "Understands systems", 'experience', 'exp', 'exp.']
            querywords = personName.split()
            
            resultwords  = [word for word in querywords if word.lower() not in stopwords]
            result = ' '.join(resultwords)   
            print(result)      
            

    """

    resume = open("resumeSample.txt", "r")
    resume_txt = resume.read()
    #print(resume_txt)
    #print(programmingScore(pdftotextmaybe.convert("Sample resumes/sample1.pdf"), jd_txt) )
    #print(extract_name(resume_txt) )
    print("Phone numbers are ",extract_phone_numbers(resume_txt) )
    print("Email id is ",extract_email_addresses(resume_txt) )


    Code to Read skills from any file 
    skills = open("skillDB.txt", "r")
    skills = skills.read()
    print(skills)
    listOfSkills = skills.split(",")
    print(listOfSkills)

    """


[nltk_data] Downloading package stopwords to C:\Users\FAITH
[nltk_data]     KATHERINE\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\FAITH
[nltk_data]     KATHERINE\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\FAITH
[nltk_data]     KATHERINE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [19]:
print(len(res_list))
res_list.sort(reverse=True)
print(res_list)

22
[48.42, 45.05, 43.98, 38.83, 36.34, 36.33, 35.71, 32.88, 32.44, 31.44, 25.85, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [20]:
df= pd.DataFrame(res_list)
df

,0
0,48.42
1,45.05
2,43.98
3,38.83
4,36.34
5,36.33
6,35.71
7,32.88
8,32.44
9,31.44


In [1]:
import nltk, re
from word2number import w2n
import pandas as pd

information=[]
inputString = ''
tokens = []
lines = []
sentences = []
max_weightage = 40;
min_variance = 5

res_list = []
dir_list = os.listdir(r'D:/FINAL YEAR PROJECT/Resumes/NewTestData/')
resumes = r'D:/FINAL YEAR PROJECT/Resumes/NewTestData/'
c = 0
for i in dir_list:
    c = c + 1
    if c == 5000:
        break
    pfinal = os.path.join(resumes, i)

    job_description = open( "D:/FINAL YEAR PROJECT/Resumes/JobDescriptions/java.txt", "r")
    job_text = job_description.read()
    #print(job_text)
    job_description.close()

    text        = job_text
    document    = pfinal

          
    try:
        # Try to get rid of special characters
        try:
            document = document.decode('ascii', 'ignore')
        except:
            #Pass as document not encoded
            pass
        # Newlines are one element of structure in the data
        # Helps limit the context and breaks up the data as is intended in resumes - i.e., into points
        lines = [el.strip() for el in re.split("\r|\n",document) if len(el) > 0]  # Splitting on the basis of newlines 
        lines = [nltk.word_tokenize(el) for el in lines]    # Tokenize the individual lines
        lines = [nltk.pos_tag(el) for el in lines]  # Tag them
        # Below approach is slightly different because it splits sentences not just on the basis of newlines, but also full stops 
        # - (barring abbreviations etc.)
        # But it fails miserably at predicting names, so currently using it only for tokenization of the whole document
        sentences = nltk.sent_tokenize(document)    # Split/Tokenize into sentences (List of strings)
        sentences = [nltk.word_tokenize(sent) for sent in sentences]    # Split/Tokenize sentences into words (List of lists of strings)
        tokens = sentences
        sentences = [nltk.pos_tag(sent) for sent in sentences]    # Tag the tokens - list of lists of tuples - each tuple is (<word>, <tag>)
        # Next 4 lines convert tokens from a list of list of strings to a list of strings; basically stitches them together
        dummy = []
        for el in tokens:
            dummy += el
        tokens = dummy
        # tokens - words extracted from the doc, lines - split only based on newlines (may have more than one sentence)
        # sentences - split on the basis of rules of grammar
        print(tokens) 
        print(lines)
        print(sentences)
    except Exception as e:
        print(e)

    expMatchStrings = ['experience', 'exp ', 'exp.', 'exp:','experience:']
    #TODO need to calculate months also
    yearStrings = ['yrs', 'years', 'yr']
    experience = []
    experience_df=pd.DataFrame(columns=('Type', 'Years', 'Months', 'Location'))
    try:
        pos = 0
        for sentence in lines:#find the index of the sentence where the degree is find and then analyse that sentence
            pos = pos+1
            sen=" ".join([words[0].lower() for words in sentence]) #string of words in sentence
            if any(re.search(x,sen) for x in expMatchStrings) and any(re.search(x,sen) for x in yearStrings):
                sen_tokenised= nltk.word_tokenize(sen)
                tagged = nltk.pos_tag(sen_tokenised)
                entities = nltk.chunk.ne_chunk(tagged)
                for subtree in entities.subtrees():
                    for leaf in subtree.leaves():
                        if leaf[1]=='CD':
                            if re.search('total',sen):
                                expType = 1
                            else: 
                                if re.search('overall',sen):
                                    expType = 2
                                else:
                                    expType = 3
                                
                            expStr = leaf[0].strip('+').strip('\x07')
                            
                            for match in (expMatchStrings+yearStrings):
                                expStr = expStr.replace(match,"")
                                
                                #If expStr contains only digit
                                try:
                                    years = float(expStr)
                                except:
                                    try:
                                        # If expStr is string which can be converted into number
                                        years = w2n.word_to_num(expStr)
                                    except:
                                        # try to remove all non-numeric characters from string except dot
                                        non_decimal = re.compile(r'[^\d.]+')
                                        expStr=non_decimal.sub("", expStr)
                                        try:
                                            years = float(expStr)
                                        except Exception as e:
                                            years = 0
                                            print(e)
                        
                                if years>0 and years < 30:
                                    experience_df = experience_df.append({'Type': expType, 'Years': years, 'Months': 0, 'Location': pos},ignore_index=True)                                    
                                    #print(experience_df)                                        
        if not experience_df.empty:
            #experience_df = experience_df.sort_values(['Type', 'Years','Location'], ascending=[True, False, True])
            experience_df = experience_df.sort_values(['Type', 'Years'], ascending=[True, False])
            experience = float(experience_df['Years'].iloc[0])
            print(experience_df)
        else:
            experience = 0.0
            print(experience)
                    
    except Exception as e:
        print (e)
    
    score = 0
    resume_exp = int(round(resume_exp))
    #print(resume_exp)
    if jd_exp.find("-") == -1:
        jd_exp = "0-"+jd_exp[:]
        
    min_jd_exp =  int(jd_exp[0])
    max_jd_exp = int(jd_exp[2])
    
    if resume_exp == 0:
        score = 0
        
    elif resume_exp > min_jd_exp:
        if resume_exp > max_jd_exp:
            score = max_weightage - (min_variance*(resume_exp-max_jd_exp))
        else:
            score = max_weightage
            
    else:
        score = max_weightage - (min_variance*(min_jd_exp-resume_exp))
        print(score)
    
    if score < 0:
        score = 0
        print(score)
        
    

['D', ':', '/FINAL', 'YEAR', 'PROJECT/Resumes/NewTestData/1.pdf']
[[('D', 'NN'), (':', ':'), ('/FINAL', 'JJ'), ('YEAR', 'NN'), ('PROJECT/Resumes/NewTestData/1.pdf', 'NNP')]]
[[('D', 'NN'), (':', ':'), ('/FINAL', 'JJ'), ('YEAR', 'NN'), ('PROJECT/Resumes/NewTestData/1.pdf', 'NNP')]]
0.0


NameError: name 'resume_exp' is not defined

In [ ]:
from gensim.models import Word2Vec
import string
import re
from nltk.corpus import stopwords
with open(r'F:/resume-ranking/skills.txt',encoding="utf8") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]

In [ ]:
content[1]

In [ ]:
from nltk.tokenize import word_tokenize
import gensim
from gensim.models.phrases import Phraser, Phrases

In [ ]:
x=[]
for line in content:
    tokens=word_tokenize(line)
    tok=[w.lower() for w in tokens]
    table=str.maketrans('','',string.punctuation)
    strpp=[w.translate(table) for w in tok]
    words=[word for word in strpp if word.isalpha()]
    stop_words=set(stopwords.words('english'))
    words=[w for w in words if not w in stop_words]
    x.append(words)

In [2]:
from resume_parser import resumeparse # Library 1
#from pyresparser import ResumeParser #Library 2
import os
import spacy
import pandas as pd
spacy.load('en_core_web_sm')

C:\Users\FAITH KATHERINE\AppData\Roaming\Python\Python37\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_training' (0.0.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [4]:
import os
from pandas.core.frame import DataFrame
from resume_parser import resumeparse # Library 1
#from pyresparser import ResumeParser #Library 2
import pandas as pd
dir_list = os.listdir(r'D:/FINAL YEAR PROJECT/Resumes/TestData/')
#print(dir_list)

res_list = []

paths = r'D:/FINAL YEAR PROJECT/Resumes/TestData/'
c = 0
for i in dir_list:
    c = c + 1
    if c == 5000:
        break
    pfinal = os.path.join(paths, i)
    #data = ResumeParser(pfinal).get_extracted_data()
    data = resumeparse.read_file(pfinal)
    res_list.append(data)
    df= pd.DataFrame(res_list)
    print(df)
   
 
    

                     email       phone              name  total_exp  \
0  dineshraja218@gmail.com  8802060884  CURRICULUM VITAE          0   

               university                  designition          degree  \
0  [annamalai university]  [quality analyst, verifier]  [M.B.A, B.B.A]   

                                              skills  \
0  [ACADEMICS:,  M.B.A (Finance) from Annamalai ...   

                                 Companies worked at  
0  [ Tally Extra Curricular Activities ,  Inte...  
                         email       phone              name  total_exp  \
0      dineshraja218@gmail.com  8802060884  CURRICULUM VITAE          0   
1  abhisheksharma268@gmail.com  9622225172   RESUME Abhishek          0   

               university                  designition          degree  \
0  [annamalai university]  [quality analyst, verifier]  [M.B.A, B.B.A]   
1                      []                           []              []   

                                    

ValueError: invalid literal for int() with base 10: '15 November 2014'

In [1]:
#Pilot Testing 
import pandas as pd
from tika import parser
from collections import Counter
import re
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize 
import io
import textract
import fitz
import os

#Dataset Loading and Preprocessing Module
def loadSkillDataset():
    skillDataset = pd.read_csv('ResumeSkill.csv')
    frontEnd = list(skillDataset['Front_End'])
    backEnd = list(skillDataset['Back_End'])
    machineLearning = list(skillDataset['Machine_Learning'])
    androidDeveloper = list(skillDataset['Android_Developer'])
    educationLevel = list(skillDataset['Education'])
    cleanedFrontEndList = [x for x in frontEnd if str(x) != 'nan']
    cleanedBackEndList = [x for x in backEnd if str(x) != 'nan']
    cleanedMachineLearningList = [x for x in machineLearning if str(x) != 'nan']
    cleanedAndroidDeveloperList = [x for x in androidDeveloper if str(x) != 'nan']
    cleanedEducationLevel = [x for x in educationLevel if str(x) != 'nan']
    return cleanedFrontEndList , cleanedBackEndList , cleanedMachineLearningList , cleanedAndroidDeveloperList, cleanedEducationLevel

frontEndList , backEndList , machineLearningList , androidDevelopmentList, educationLevelList = loadSkillDataset()

dir_list = os.listdir(r'D:/FINAL YEAR PROJECT/Resumes/NewTestData/')
resumes=r'D:/FINAL YEAR PROJECT/Resumes/NewTestData/'
# Resume File Text Extractor Module
for i in dir_list:
    #c = c + 1
    #if c == 5000:
        #break
    pfinal = os.path.join(resumes, i)

    #def extract_text_from_pdf(file):
    def fileTextExtractor():
        '''Opens and reads in a PDF file from path'''
        resume_text=''
        if i.endswith('.pdf'):
            doc = fitz.open(pfinal)
            
            for page in doc:
                resume_text = resume_text + str(page.getText())
            resume_text = "".join(resume_text.split("\n"))
        
        '''Opens en reads in a .doc or .docx file from path'''

        if i.endswith('.doc'):
            resume_text = textract.process(pfinal).decode('utf-8')
        return resume_text

    obtainedResumeText = fileTextExtractor()



#Resume email and phone number extractor Module
    def personalDetailExtractor():
        finalExtractedEmail = []
        finalExtractedPhone = []
        oneFourthOfResume = obtainedResumeText[0:len(obtainedResumeText)//4] 
        emailResume = re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+", oneFourthOfResume)
        phoneResume = re.findall(r"(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})",oneFourthOfResume)
        
        if len(emailResume) > 1:
            finalExtractedEmail = emailResume[0]
        else:
            finalExtractedEmail = emailResume
            
        for i in range(len(phoneResume)):
            if len(phoneResume[i])>=10:
                finalExtractedPhone = phoneResume[i]
        return finalExtractedEmail,finalExtractedPhone

    finalExtractedEmail , finalExtractedPhone = personalDetailExtractor()

    #Capitalizing, LowerCase and Upper Case Conversion of the Resume Text Module
    firstLetterCapitalizedObtainedResumeText = []
    def CapitalizeFirstLetter(obtainedResumeText):
        capitalizingString = " "
        obtainedResumeTextLowerCase = obtainedResumeText.lower()
        obtainedResumeTextUpperCase = obtainedResumeText.upper()
        splitListOfObtainedResumeText = obtainedResumeText.split()
        for i in splitListOfObtainedResumeText:
            firstLetterCapitalizedObtainedResumeText.append(i.capitalize())        
        return (capitalizingString.join(firstLetterCapitalizedObtainedResumeText),obtainedResumeTextLowerCase,obtainedResumeTextUpperCase)
    firstLetterCapitalizedText,obtainedResumeTextLowerCase,obtainedResumeTextUpperCase = CapitalizeFirstLetter(obtainedResumeText)

    #Combination of LowerCase UpperCase and FirstLetter Capitalized
    obtainedResumeText = obtainedResumeTextLowerCase + obtainedResumeTextUpperCase + firstLetterCapitalizedText
    # Removing numbers from text file
    obtainedResumeText = re.sub(r'\d+','',obtainedResumeText)
    # Remove punctuation from the text files
    obtainedResumeText = obtainedResumeText.translate(str.maketrans('','',string.punctuation))

    #Resume educational Details Extracting Module
    def EducationDetailsExtractor(obtainedResumeText):
        obtainedResumeText.strip('/n')
        newLineRemovedResumeText = obtainedResumeText    
        resumeEducationSpecificationList = {'Education':educationLevelList}

        # Create an empty list where the scores will be stored
        educationExtracted = []
        for area in resumeEducationSpecificationList.keys():
            if area == 'Education' or 'Highschool' or 'Bachelors' or 'Bsc.':
                educationWord = []
                for word in resumeEducationSpecificationList[area]:
                    if word in obtainedResumeText:
                        educationWord.append(word)
                educationExtracted.append(educationWord)
        return educationExtracted

    extractedEducatioDetails = EducationDetailsExtractor(obtainedResumeText)

    def stopWordRemoval(obtainedResumeText):
        stop_words = set(stopwords.words('english')) 
        word_tokens = word_tokenize(obtainedResumeText) 
        filtered_sentence = [w for w in word_tokens if not w in stop_words] 

        filtered_sentence = [] 
        joinEmptyString = " "
        for w in word_tokens: 
            if w not in stop_words: 
                filtered_sentence.append(w)
        return(joinEmptyString.join(filtered_sentence))
        
    filteredTextForSkillExtraction = stopWordRemoval(obtainedResumeText)

    #Resume Skill Specification Listing and Extracting Module
    resumeTechnicalSkillSpecificationList = {'Front End':frontEndList,

                'Back End':backEndList, 'Machine Learning':machineLearningList,'Android Developer':androidDevelopmentList}

    def ResumeSkillExtractor(resumeTechnicalSkillSpecificationList,filteredTextForSkillExtraction):
        frontend = 0
        backend = 0
        machinelearning = 0
        androiddeveloper = 0

        # Create an empty list where the scores will be stored
        skillScores = []
        skillExtracted = []


        # Obtain the scores for each area
        for area in resumeTechnicalSkillSpecificationList.keys():

            if area == 'Front End':
                frontEndWord = []
                for word in resumeTechnicalSkillSpecificationList[area]:
                    if word in filteredTextForSkillExtraction:
                        frontend += 1
                        frontEndWord.append(word)
                skillExtracted.append(frontEndWord)
                skillScores.append(frontend)

            elif area == 'Back End':
                backEndWord = []
                for word in resumeTechnicalSkillSpecificationList[area]:
                    if word in filteredTextForSkillExtraction:
                        backend += 1
                        backEndWord.append(word)
                skillExtracted.append(backEndWord)
                skillScores.append(backend)

            elif area == 'Machine Learning':
                machineLearningWord = []
                for word in resumeTechnicalSkillSpecificationList[area]:
                    if word in filteredTextForSkillExtraction:
                        machinelearning += 1
                        machineLearningWord.append(word)
                skillExtracted.append(machineLearningWord)
                skillScores.append(machinelearning)

            elif area == 'Android Developer':
                androidDeveloperWord = []
                for word in resumeTechnicalSkillSpecificationList[area]:
                    if word in filteredTextForSkillExtraction:
                        androiddeveloper += 1
                        androidDeveloperWord.append(word)
                skillExtracted.append(androidDeveloperWord)
                skillScores.append(androiddeveloper)
        return skillScores,skillExtracted
    technicalSkillScore , technicalSkillExtracted = ResumeSkillExtractor(resumeTechnicalSkillSpecificationList,filteredTextForSkillExtraction)


    '''Personal Details, Skills and Academic Qualification Output Display'''

    dataList = {'Scores':technicalSkillScore,"Skills":technicalSkillExtracted}
    softwareDevelopemtTechnicalSkills = pd.DataFrame(dataList,index=resumeTechnicalSkillSpecificationList.keys())
    print("Email Address:",finalExtractedEmail)
    print("Phone Number:",finalExtractedPhone)
    print("Academic Qualifications:",extractedEducatioDetails)
    print(softwareDevelopemtTechnicalSkills)
    df = pd.DataFrame({"Email":[finalExtractedEmail], "Phone":[finalExtractedPhone], 
    "Academic Qualifications":[extractedEducatioDetails], "Skills":[softwareDevelopemtTechnicalSkills]})
    print(df)

Email Address: ['mikewanaswa@gmail.com']
Phone Number: []
Academic Qualifications: [[]]
                   Scores  Skills
Front End               1  [LESS]
Back End                1   [ORM]
Machine Learning        1    [R ]
Android Developer       1    [GO]
                     Email Phone Academic Qualifications  \
0  [mikewanaswa@gmail.com]    []                    [[]]   

                                              Skills  
0                     Scores  Skills
Front End   ...  
Email Address: ['shub.pandey7@yahoo.in']
Phone Number: 8115031253
Academic Qualifications: [['Bachelor']]
                   Scores  Skills
Front End               1  [LESS]
Back End                1   [ORM]
Machine Learning        1    [R ]
Android Developer       1    [GO]
                     Email       Phone Academic Qualifications  \
0  [shub.pandey7@yahoo.in]  8115031253            [[Bachelor]]   

                                              Skills  
0                     Scores  Skills
Front End 